In [17]:
import math, sys
class BayesianFilter:
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.word_dict = {} # 카테고리마다의 출현 횟수 기록
        self.category_dict = {} # 카테고리 출현 횟수 기록
    # 형태소 분석하기 --- (※1)
    def split(self, text):
        results = text.split()
        return results
    # 단어와 카테고리의 출현 횟수 세기 --- (※2)
    def inc_word(self, word, category):
        # 단어를 카테고리에 추가하기
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)
    def inc_category(self, category):
        # 카테고리 계산하기
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1

    # 텍스트 학습 --- (※3)
    def fit(self, text, category):
        word_list = self.split(text)
        for word in word_list:
            self.inc_word(word, category)
        self.inc_category(category)
    
    # 단어 리스트에 점수 매기기--- (※4)
    def score(self, words, category):
        score = math.log(self.category_prob(category))
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score
    
    # 예측 --- (※5)
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize 
        words = self.split(text)
        score_list = []
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list

    # 카테고리 내부의 단어 출현 비율 계산 --- (※6)
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1 # ---(※6a)
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d
    
    # 카테고리 내부의 단어 출현 횟수 구하기
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0
    # 카테고리 계산
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories


In [18]:

bf = BayesianFilter()
# 텍스트 학습
bf.fit("파격 세일 - 오늘까지만 30% 할인", "광고")
bf.fit("쿠폰 선물 & 무료 배송", "광고")
bf.fit("현데계 백화점 세일", "광고")
bf.fit("봄과 함께 찾아온 따뜻한 신제품 소식", "광고")
bf.fit("인기 제품 기간 한정 세일", "광고")
bf.fit("오늘 일정 확인", "중요")
bf.fit("프로젝트 진행 상황 보고","중요")
bf.fit("계약 잘 부탁드립니다","중요")
bf.fit("회의 일정이 등록되었습니다.","중요")
bf.fit("오늘 일정이 없습니다.","중요")
# 예측
pre, scorelist = bf.predict("재고 정리 할인, 무료 배송")
print("결과 =", pre)
print(scorelist)

결과 = 광고
[('광고', -19.942524744665512), ('중요', -20.544606748320554)]


In [19]:
print(bf.word_dict)

{'광고': {'파격': 1, '세일': 3, '-': 1, '오늘까지만': 1, '30%': 1, '할인': 1, '쿠폰': 1, '선물': 1, '&': 1, '무료': 1, '배송': 1, '현데계': 1, '백화점': 1, '봄과': 1, '함께': 1, '찾아온': 1, '따뜻한': 1, '신제품': 1, '소식': 1, '인기': 1, '제품': 1, '기간': 1, '한정': 1}, '중요': {'오늘': 2, '일정': 1, '확인': 1, '프로젝트': 1, '진행': 1, '상황': 1, '보고': 1, '계약': 1, '잘': 1, '부탁드립니다': 1, '회의': 1, '일정이': 2, '등록되었습니다.': 1, '없습니다.': 1}}


In [20]:
print(bf.category_dict)

{'광고': 5, '중요': 5}


# 모든 문장을 받아 한번에 학습하는 sfit() 함수를 작성하세요. nltk.ConditionalFreqDist()을 사용해도 됨. 

In [25]:
import nltk
import math, sys
class BayesianFilter1:
    """ 베이지안 필터 """
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.categories = {} # 카테고리 출현 횟수 기록
        self.cfd = _
 
    # 텍스트 문장 학습하기 --- (※3)
    def sfit(self, data):
        """ 텍스트 학습 """
        train_data = []
        for s, c in data:
            for w in s.split():
                train_data.append((c, w))
                if c in self.categories :
                    self.categories[c] += 1
                else:
                    self.categories[c] = 1
                self.words.add(w)
        self.cfd = nltk.ConditionalFreqDist(train_data)

    # 텍스트 문장 학습하기 --- (※3)
    def lfit(self, data):
        """ 텍스트 학습 """
        train_data = []
        for s, c in data:
            for w in s:
                train_data.append((c, w))
                if c in self.categories :
                    self.categories[c] += 1
                else:
                    self.categories[c] = 1
                self.words.add(w)
        self.cfd = nltk.ConditionalFreqDist(train_data)
        
    # 텍스트 단어 학습하기 --- (※3)
    def fit(self, data):
        """ 텍스트 학습 """
        train_data = []
        for w, c in data:
            train_data.append((c, w))
            if c in self.categories :
                self.categories[c] += 1
            else:
                self.categories[c] = 1
            self.words.add(w)
        self.cfd = nltk.ConditionalFreqDist(train_data)

    # 단어 리스트에 점수 매기기--- (※4)
    def score(self, words, category):
        s_category = sum(self.categories.values())         
        score = math.log( self.categories[category] / s_category )
        for word in words:
            score += math.log( (self.cfd[category][word]+1) / (self.categories[category] + len(self.words)) )
        return score
    
    # 예측하기 --- (※5)
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize 
        words = text.split()
        score_list = []

        for category in self.categories:
            score = self.score(words, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list


In [22]:
# 텍스트 학습
train = (
("파격 세일 - 오늘까지만 30% 할인", "광고"), 
("쿠폰 선물 & 무료 배송", "광고"), 
("현데계 백화점 세일", "광고"), 
("봄과 함께 찾아온 따뜻한 신제품 소식", "광고"), 
("인기 제품 기간 한정 세일", "광고"),
("오늘 일정 확인", "중요"),
("프로젝트 진행 상황 보고","중요"),
("계약 잘 부탁드립니다","중요"),
("회의 일정이 등록되었습니다.","중요"),
("오늘 일정이 없습니다.","중요"))

bf = BayesianFilter1()
bf.sfit( train) # sentence fit
# 예측
pre, scorelist = bf.predict("재고 정리 할인, 무료 배송")
print("결과 =", pre)
print(scorelist)
print(bf.categories)

결과 = 광고
[('광고', -19.744073805941674), ('중요', -20.792442912225134)]
{'광고': 25, '중요': 16}


# brown corpus

In [14]:
from nltk.corpus import brown

print(brown.categories())

['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies', 'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance', 'science_fiction']


In [23]:
genre_sents = [(s, genre) 
               for genre in ['news', 'romance']
               for s in brown.sents(categories=genre)] 
print(genre_sents)

[(['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], 'news'), (['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], 'news'), (['The', 'September-October', 'term', 'jury', 'had', 'been', 'charged', 'by', 'Fulton', 'Superior', 'Court', 'Judge', 'Durwood', 'Pye', 'to', 'investigate', 'reports', 'of', 'possible', '``', 'irregularities', "''", 'in', 'the', 'hard-fought', 'primary', 'which', 'was', 'won', 'by', 'Mayor-nominate', 'Ivan', 'Allen', 'Jr.', '.'], 'news'), (['``', 'Only', 'a', 'relative'

In [26]:
bf = BayesianFilter1()
bf.lfit( genre_sents) # sentence fit
# 예측
pre, scorelist = bf.predict("Good luck to you all the best")
print("결과 =", pre)
print(scorelist)
print(bf.categories)

결과 = romance
[('news', -51.24489975442419), ('romance', -47.46630612000449)]
{'news': 100554, 'romance': 70022}
